### Import Libraries

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium

print('Libraries imported.')

Libraries imported.


### Connect to our Dataset

Our dataset is available for free on the internet. Feel free to use it yourself: https://cocl.us/new_york_dataset'

In [11]:
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()

### Let's transform the data into a Pandas Dataframe

In [13]:
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    
    # all data is present in features label
    features=resp['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    return new_york_data

In [14]:
neighborhoods = get_new_york_data()
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Let's get the geographical coordinates of New York

In [15]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Create a map of New York with neighborhoods on top

In [17]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [21]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Explore Foursquare Venues

In [22]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [23]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'IE0US3KJVFKX0VRUAULZTY5W0ZLRCYBCDDRMZM0HI50RLSG2'
CLIENT_SECRET = 'LQA2JNDHI4NGQQWU2LWILWBQAK5P0D4I1YZLJIJJO04HXAW1'
VERSION = '20181020'

In [24]:
# Link: https://developer.foursquare.com/docs/resources/categories
#GYM = 4bf58dd8d48988d176941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_gym = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d176941735')
newyork_venues_gym.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Blink Fitness,40.877271,-73.905595,Gym
1,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym
2,Marble Hill,40.876551,-73.91066,3210 Riverdale Avenue - Wellness Center & Gym,40.882746,-73.907625,Gym
3,Marble Hill,40.876551,-73.91066,TCR The Club of Riverdale,40.878628,-73.914568,Tennis Stadium
4,Marble Hill,40.876551,-73.91066,The Arbor Gym,40.885689,-73.912437,Gym


In [25]:
newyork_venues_gym.shape


(1696, 7)

In [26]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

### Creating a map with the GYMs in New York

In [27]:
map_newyork_gym = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_gym, 'red', map_newyork_gym)

map_newyork_gym

In [28]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [29]:
manhattan_grouped = newyork_venues_gym.groupby('Neighborhood').count()
manhattan_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,49,49,49,49,49,49
Carnegie Hill,49,49,49,49,49,49
Central Harlem,25,25,25,25,25,25
Chelsea,50,50,50,50,50,50
Chinatown,50,50,50,50,50,50
Civic Center,49,49,49,49,49,49
Clinton,49,49,49,49,49,49
East Harlem,39,39,39,39,39,39
East Village,49,49,49,49,49,49


In [31]:
print('There are {} uniques categories.'.format(len(newyork_venues_gym['Venue Category'].unique())))

There are 25 uniques categories.


### Analyze each neighbourhood

In [33]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_gym[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_gym['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Basketball Court,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center,Dance Studio,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Martial Arts School,Office,Outdoor Gym,Pilates Studio,Playground,Pool,Recreation Center,Residential Building (Apartment / Condo),School,Spa,Tennis Court,Tennis Stadium,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Marble Hill,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Basketball Court,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center,Dance Studio,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Martial Arts School,Office,Outdoor Gym,Pilates Studio,Playground,Pool,Recreation Center,Residential Building (Apartment / Condo),School,Spa,Tennis Court,Tennis Stadium,Yoga Studio
0,Battery Park City,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.857143,0.102041,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.020408
1,Carnegie Hill,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.836735,0.081633,0.020408,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.020408
2,Central Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.960000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Chelsea,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.800000,0.140000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.0,0.020000
4,Chinatown,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.780000,0.100000,0.000000,0.000000,0.060000,0.020000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,Civic Center,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.734694,0.163265,0.000000,0.000000,0.040816,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.020408
6,Clinton,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.795918,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.0,0.020408
7,East Harlem,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.871795,0.051282,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.025641
8,East Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.755102,0.163265,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.020408,0.000000,0.0,0.020408
9,Financial District,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.877551,0.102041,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Gym,Gym / Fitness Center,Yoga Studio,Corporate Amenity,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Cultural Center
1,Carnegie Hill,Gym,Gym / Fitness Center,Yoga Studio,Community Center,Pilates Studio,Gym Pool,Gymnastics Gym,Building,Clothing Store,Club House
2,Central Harlem,Gym,Gym / Fitness Center,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center
3,Chelsea,Gym,Gym / Fitness Center,Yoga Studio,Residential Building (Apartment / Condo),Martial Arts School,School,Recreation Center,Building,Clothing Store,Club House
4,Chinatown,Gym,Gym / Fitness Center,Martial Arts School,Clothing Store,Pool,Office,Gym Pool,Building,Club House,Community Center


### Cluster neighbourhoods

In [37]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 4, 4, 0, 4, 1, 0, 1], dtype=int32)

In [48]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Gym,Gym / Fitness Center,Tennis Stadium,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity
1,Manhattan,Chinatown,40.715618,-73.994279,4,Gym,Gym / Fitness Center,Martial Arts School,Clothing Store,Pool,Office,Gym Pool,Building,Club House,Community Center
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Gym,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center,Dance Studio
3,Manhattan,Inwood,40.867684,-73.921210,1,Gym,Gym / Fitness Center,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Gym,Basketball Court,Gym / Fitness Center,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center


In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln Square,Gym,Gym / Fitness Center,Yoga Studio,Residential Building (Apartment / Condo),Cultural Center,Pilates Studio,Gym Pool,Building,Clothing Store,Club House
15,Midtown,Gym,Gym / Fitness Center,Yoga Studio,Building,Residential Building (Apartment / Condo),Gymnastics Gym,Clothing Store,Club House,Community Center,Corporate Amenity
19,East Village,Gym,Gym / Fitness Center,Yoga Studio,Spa,Pool,Outdoor Gym,Gym Pool,Building,Clothing Store,Club House
21,Tribeca,Gym,Gym / Fitness Center,Martial Arts School,Office,Corporate Amenity,Yoga Studio,Recreation Center,Spa,Building,Clothing Store
23,Soho,Gym,Gym / Fitness Center,Yoga Studio,Martial Arts School,Office,School,Spa,Building,Clothing Store,Club House
27,Gramercy,Gym,Gym / Fitness Center,Yoga Studio,Pool,Outdoor Gym,Martial Arts School,Spa,Residential Building (Apartment / Condo),Tennis Court,Building
32,Civic Center,Gym,Gym / Fitness Center,Martial Arts School,Office,Corporate Amenity,Yoga Studio,Recreation Center,Spa,Building,Clothing Store


In [51]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Gym,Gym / Fitness Center,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center
4,Hamilton Heights,Gym,Basketball Court,Gym / Fitness Center,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center
7,East Harlem,Gym,Gym / Fitness Center,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center
8,Upper East Side,Gym,Gym / Fitness Center,Yoga Studio,Gym Pool,Club House,Recreation Center,Pilates Studio,Building,Clothing Store,Community Center
9,Yorkville,Gym,Gym / Fitness Center,Yoga Studio,Community Center,Pilates Studio,Gym Pool,Gymnastics Gym,Building,Clothing Store,Club House
12,Upper West Side,Gym,Gym / Fitness Center,Yoga Studio,Cultural Center,Playground,Pilates Studio,Gym Pool,Building,Clothing Store,Club House
24,West Village,Gym,Gym / Fitness Center,Spa,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity
25,Manhattan Valley,Gym,Gym / Fitness Center,Playground,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity
28,Battery Park City,Gym,Gym / Fitness Center,Yoga Studio,Corporate Amenity,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Cultural Center
29,Financial District,Gym,Gym / Fitness Center,Corporate Amenity,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Cultural Center


In [52]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Gym,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center,Dance Studio
5,Manhattanville,Gym,Basketball Court,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center,Dance Studio
6,Central Harlem,Gym,Gym / Fitness Center,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center
20,Lower East Side,Gym,Gym / Fitness Center,Clothing Store,Pool,Yoga Studio,Gymnastics Gym,Building,Club House,Community Center,Corporate Amenity
26,Morningside Heights,Gym,Gym / Fitness Center,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center
37,Stuyvesant Town,Gym,Martial Arts School,Outdoor Gym,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity,Cultural Center


In [53]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Murray Hill,Gym,Gym / Fitness Center,Yoga Studio,Building,Martial Arts School,Residential Building (Apartment / Condo),Recreation Center,Tennis Court,Clothing Store,Club House
18,Greenwich Village,Gym,Gym / Fitness Center,Martial Arts School,Office,Yoga Studio,Spa,Recreation Center,Building,Clothing Store,Club House
22,Little Italy,Gym,Gym / Fitness Center,Yoga Studio,Martial Arts School,Office,School,Spa,Building,Clothing Store,Club House
33,Midtown South,Gym,Gym / Fitness Center,Residential Building (Apartment / Condo),Yoga Studio,Building,Martial Arts School,Recreation Center,Spa,Tennis Court,Clothing Store
38,Flatiron,Gym,Gym / Fitness Center,Yoga Studio,Residential Building (Apartment / Condo),Pool,Martial Arts School,Spa,Tennis Court,Building,Clothing Store


In [54]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Gym,Gym / Fitness Center,Tennis Stadium,Yoga Studio,Gymnastics Gym,Building,Clothing Store,Club House,Community Center,Corporate Amenity
1,Chinatown,Gym,Gym / Fitness Center,Martial Arts School,Clothing Store,Pool,Office,Gym Pool,Building,Club House,Community Center
10,Lenox Hill,Gym,Gym / Fitness Center,Tennis Court,Club House,Recreation Center,Yoga Studio,Gym Pool,Building,Clothing Store,Community Center
11,Roosevelt Island,Gym,Gym / Fitness Center,Tennis Court,School,Recreation Center,Club House,Dance Studio,Yoga Studio,Gym Pool,Building
14,Clinton,Gym,Gym / Fitness Center,Yoga Studio,Building,Residential Building (Apartment / Condo),Gymnastics Gym,Clothing Store,Club House,Community Center,Corporate Amenity
17,Chelsea,Gym,Gym / Fitness Center,Yoga Studio,Residential Building (Apartment / Condo),Martial Arts School,School,Recreation Center,Building,Clothing Store,Club House
31,Noho,Gym,Gym / Fitness Center,Yoga Studio,Pool,Clothing Store,Outdoor Gym,Martial Arts School,Recreation Center,Playground,Building
34,Sutton Place,Gym,Gym / Fitness Center,Tennis Court,Residential Building (Apartment / Condo),Club House,Yoga Studio,Gym Pool,Building,Clothing Store,Community Center
39,Hudson Yards,Gym,Gym / Fitness Center,Building,Residential Building (Apartment / Condo),Yoga Studio,Gymnastics Gym,Clothing Store,Club House,Community Center,Corporate Amenity


### Thanks. OK.